# OLD - Notebook from July/August 2022. I used it for creating the prior-block function

- A phase where I got to know numba (In the end I use it especially for the anomaly - mean calculations)

This notebook contains code_snippets for:

- creating the priorblock depending on the maximum time resolution
- Computing means and anomalies over these blocks and its x-year sublocks

They are tested here before being used as functions in the final Dataassimilation wrapper

Efficiency:

  - The priorblock is created in each MonteCarlo repetition instead of once in the beginning, because accesstimes would be slow anyways when havin a really large priorblock containing all possible ensemble members
  - Mean and Annomaly calculation: 
      - For assimilating the x-year mean in the multitimescale approach, we need to compute averages and annomalies over very large arrays (gridpoints*ensemblemembers) over and over again    
      - computing the annomalies is more costly than the simple average
      - Along the blocksize-axis (e.g. length 20) we need to calculate the 2,5,10... year averages, so we need to split the array/go through the array section
        
      - **np.mean is not parallelized** which would make sense for arrays of length ~10⁶ that we have. The calculations on the long axis are independent from each other. I therefore implemented the mean and anomaly calculation with a naive for loop and numba @njit(parallel=True
        
      - On my current machine this gives an improvement of 4-10 with respect to the numpy solution, on the new machine numpy on one core is much more efficient, such that the time-advantage is only 1.5 - 2. Curiously, the parallelized code is a bit faster on the old machine (5-10ms).
      - The new machine is improves copying large arrays by a factor of ~ 5

Test-Data:
* iCESM has the highest spatial resolution (192*94 = 18048 gridpoints). So we test all algorithms on that data. Probably improvements in annomaly calculation are better for this model than for others

**Further improvements in the future:**
- Initially I though of using Dask for this task, but it didn't seem to speed up the calculation
- I asked for advice on Stackoverflow: https://stackoverflow.com/questions/73214684/optimize-computation-of-anomaly-from-mean-along-one-axis-for-large-arrays
And not so surprisingly the answer I got suggested using tensorflow on a GPU if I can
- I could probably transition parts of my DA code (the actual Kalman Filter parts) to tensorflow, but one would need to check if all operations actually profit of this.
- As all I do in the end is only linear Algebra, I think it should be possible to try this out, but it would require some testing and learning how the loading on the GPU overhead can be minimized




In [202]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

#ENABLE SMOOTH SCROLLING

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [201]:
%%javascript
Jupyter.keyboard_manager.command_shortcuts.remove_shortcut('up');
Jupyter.keyboard_manager.command_shortcuts.remove_shortcut('down');

<IPython.core.display.Javascript object>

In [3]:
import sys
sys.path.append('/home/mchoblet/paleoda')

import xarray as xr
import numpy as np

import dataloader
from numba import jit,njit,prange


In [4]:
#Test data is from ccsm
ccsm=xr.open_dataset('/home/mchoblet/paleoda/data/model_data/CCSM_d18O_851_1850.nc')
ccsm_m=dataloader.annual_mean(ccsm['d18O'])
#load d18O and stack it
d18O_=ccsm_m.stack(z=('lat','lon')).values

# Block creation

We ad a new axis with the blocksize as size, such that the stacked vectors with size (Ne,Nx) become (N_bs,Ne,Nx).

For facilitating things we go towards using pure numpy arrays instead of xarrays.

In [5]:
bs=20
reps=5
nens=100
MC_idx=dataloader.random_indices(nens,ccsm_m.time.shape[0]-bs-1,reps,seed=None)


In [6]:
#reduce d18O to the years for this MonteCarlo-Index 
rep=0
d18O_red=d18O_[MC_idx[rep],:]
print('Old shape',d18O_.shape)
print('New shape',d18O_red.shape)


Old shape (1000, 18048)
New shape (100, 18048)


In [9]:
%%time


prior_block=np.empty((bs,nens,d18O_red.shape[1]))

l=[]
for i in range(bs):
    prior_block[i]=d18O_[MC_idx[rep]+i,:]
    #l.append(d18O_[MC_idx[rep]+i,:])

CPU times: user 29.2 ms, sys: 9.64 ms, total: 38.8 ms
Wall time: 37.9 ms


In [10]:
%%time
block=np.empty((bs,nens,18048))

CPU times: user 24 µs, sys: 8 µs, total: 32 µs
Wall time: 35.5 µs


In [11]:
def prior_block_nonumba(prior,nens,stack_size,bs,MC_idx):
    block=np.empty((bs,nens,stack_size))
    for i in range(bs):
        block[i]=prior[MC_idx+i,:]
    return block

In [12]:
@njit(parallel=True,fastmath=True)
def prior_block(prior,nens,stack_size,bs,MC):
    block=np.empty((bs,nens,stack_size))
    for i in prange(bs):
        #for j in prange(nens):
        #M=MC+i
        for k in prange(stack_size):
            for j in prange(nens):
                block[i,j,k]=prior[MC[j]+i,k]
    return block

In [14]:
%%timeit  -n 10 -r 10
b=prior_block(d18O_,100,18048,20,MC_idx[0])

32.1 ms ± 49.5 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [15]:
%%timeit  -n 10 -r 10
b2=prior_block_nonumba(d18O_,100,18048,20,MC_idx[0])

34.4 ms ± 323 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [11]:
@njit(parallel=True,fastmath=True)
def prior_block_stack(prior,nens,stack_size,bs,MC_idx):
    block=[]
    for i in prange(bs):
        block.append(prior[MC_idx+i,:])
    return block

In [18]:
%%time
be=prior_block_stack(d18O_,100,18048,20,MC_idx[0])
b=np.stack(be)

CPU times: user 206 ms, sys: 27.9 ms, total: 234 ms
Wall time: 81.5 ms


# Means and Annomalies over prior block

* We reshape the prior block to (bs,gridpoints*nens), and do the operations over axis0. Reshaping is a matter of nanoseconds

The link to my Stackoverflow question:
https://stackoverflow.com/questions/73214684/optimize-computation-of-anomaly-from-mean-along-one-axis-for-large-arrays

- I also though of using np.add.reduceat (https://stackoverflow.com/questions/53178018/average-of-elements-in-a-subarray, response with 4 upvotes) for vectorising my code, but i dont think it is more efficient. Naive intents of vectorising the computation did not bring benefits. Up to now the numba solution is the best i came up with

- Numba discussion for axis keyword in np mean: https://github.com/numba/numba/issues/1269


In [52]:
%%timeit
prior_block_res=prior_block.reshape(20,-1)

139 ns ± 0.0658 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [142]:
prior_block_res=prior_block.reshape(20,-1)

In [56]:
%%timeit
prior_ano=prior_block_res-prior_block_res.mean(axis=0)

279 ms ± 39.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [207]:
%%time
test_array=np.random.randn(20,10000000)

CPU times: user 4.62 s, sys: 1.24 s, total: 5.86 s
Wall time: 5.86 s


In [218]:
%%time
happyment=numpy_with_numba(test_array,5)

CPU times: user 813 ms, sys: 438 ms, total: 1.25 s
Wall time: 227 ms


In [219]:
%%time
numpy_no_numba(test_array,5)

CPU times: user 743 ms, sys: 3.57 s, total: 4.31 s
Wall time: 4.31 s


(array([[ 0.3111084 ,  0.63536053, -0.65521184, ...,  0.6899033 ,
          0.50454104,  0.28147666],
        [ 0.09007034,  0.20041499, -0.01540553, ..., -1.01182403,
         -0.19205175,  0.13042424],
        [ 0.13937721, -0.18388498, -0.36384981, ..., -0.54554122,
         -0.19268758,  0.59810223],
        [-0.6169305 , -0.2970972 , -0.39862908, ...,  0.37782842,
         -0.41409898, -0.99749505]]),
 array([[-0.83596571,  0.37944479,  0.6115543 , ..., -0.56755288,
          1.38715909, -1.05460021],
        [ 0.890776  , -0.15011219, -0.31720856, ..., -1.01271152,
          0.29433425, -0.62255085],
        [-1.15514587, -0.12786244, -0.28109493, ...,  0.54739501,
         -0.66259425,  1.90162982],
        ...,
        [-1.01348696, -0.30950701,  0.13409921, ...,  0.04958339,
         -0.06160397,  0.0044071 ],
        [-0.21627612, -0.58708353,  0.37779695, ..., -0.30278231,
         -0.46113333, -0.30778193],
        [ 1.35084141, -1.09833028, -1.11693149, ...,  1.2220668 ,
 

In [139]:
def numpy_no_numba(array_in,size):
    s=array_in.shape
    #number of subarrays along axis0
    number=s[0]//size

    mean=np.empty((number,s[1]))
    anom=np.empty_like(array_in)

    #loop over the subarrays
    for i in range(number):
        block=array_in[i*size:(i+1)*size,:]
        block_mean=block.mean(axis=0)
        mean[i]=block_mean
        anom[i*size:(i+1)*size,:]=block-block_mean
    
    return mean,anom

@njit(parallel=True)
def numpy_with_numba(array_in,size):

    s=array_in.shape
    number=s[0]//size
    
    #Initialize array for mean and anomaly (latter has original size)
    mean=np.empty((number,s[1]))
    anom=np.empty_like(array_in)
    
    #looping over blocks if size!=s[0]
    if number>1:
        #loop over all points along axis1
        for i in prange(s[1]):
            for j in prange(number):
                vals=array_in[j*size:(j+1)*size,i]
                m=vals.mean()
                mean[j,i]=m
                anom[j*size:(j+1)*size,i]=vals-m
    else:
        for i in prange(s[1]):
            vals=array_in[:,i]
            m=vals.mean()
            mean[0,i]=m
            anom[:,i]=vals-m
            
    return mean,anom


In [104]:

def numpy_no_numba_split(array_in,size):
    """
    Input:
        array: numpy array of size (A,B,C,...). A is the axis where the mean is taken
        size: Integer which denominates size of block (along axis0) over which annomaly and mean is calcuation
    """
    s=array_in.shape
    #number of subarrays along axis0
    number=s[0]//size
       
    if number!=1:
        #Initialize array for mean and annomalies 
        mean=np.empty((number,s[1]))
        anom=np.empty((number,size,s[1]))
        #split input array along axis0
        array_in_split=np.split(array_in,number,axis=0)

        for i,a in enumerate(array_in_split):
            m=a.mean(axis=0)
            mean[i]=m
            anom[i]=a-m
        
    else:
        mean=array_in.mean(axis=0)
        anom=array_in - mean
        
    return mean,anom.reshape(s[0],-1)

In [106]:
%%timeit -r 10 -n 10
m1,a1=numpy_no_numba_split(prior_block_res,2)

323 ms ± 31.2 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [107]:
%%timeit -r 10 -n 10
m1,a1=numpy_no_numba(prior_block_res,2)

330 ms ± 15 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [108]:
%%timeit -r 10 -n 10
m1,a1=numpy_with_numba(prior_block_res,2)

53.8 ms ± 426 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [112]:
%%timeit -r 10 -n 10
m1,a1=numpy_no_numba_split(prior_block_res,5)

394 ms ± 18 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [113]:
%%timeit -r 10 -n 10
m1,a1=numpy_no_numba(prior_block_res,5)

375 ms ± 4.49 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [114]:
%%timeit -r 10 -n 10
m1,a1=numpy_with_numba(prior_block_res,5)

42.6 ms ± 467 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [115]:
%%timeit -r 10 -n 10
m1,a1=numpy_no_numba_split(prior_block_res,10)

343 ms ± 18.3 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [116]:
%%timeit -r 10 -n 10
m1,a1=numpy_no_numba(prior_block_res,10)

338 ms ± 6.01 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [117]:
%%timeit -r 10 -n 10
m1,a1=numpy_with_numba(prior_block_res,10)

36.7 ms ± 536 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [150]:
%%timeit
### ADD Back the mean

m1,a1=numpy_with_numba(prior_block_res,10)
new_prior=a1[:10]+m1[0]
new_prior=a1[10:]+m1[1]

88 ms ± 471 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [164]:
%%time
m1,a1=numpy_with_numba(prior_block_res,20)
new_prior=m1+a1

CPU times: user 153 ms, sys: 124 ms, total: 277 ms
Wall time: 92.4 ms


In [198]:
%%time
m1,a1=numpy_with_numba(prior_block_res,10)
b[:10]=a1[:10]+m1[0]
b[10:20]=a1[10:]+m1[1]

CPU times: user 195 ms, sys: 132 ms, total: 327 ms
Wall time: 111 ms


In [182]:
b=a1.copy()

In [199]:
%%time
m1,a1=numpy_with_numba(prior_block_res,5)
b[:5]=a1[:5]+m1[0]
b[5:10]=a1[5:10]+m1[1]
b[10:15]=a1[10:15]+m1[1]
b[15:20]=a1[15:20]+m1[1]

CPU times: user 215 ms, sys: 125 ms, total: 340 ms
Wall time: 114 ms


In [181]:
b

<function ndarray.copy>

In [146]:
a1.shape

(20, 1804800)

In [88]:
a=[prior_block_res+i for i in range(100)]

In [90]:
%%time
for i in range(100):
    numpy_no_numba(a[i],5)

CPU times: user 10.7 s, sys: 12.7 s, total: 23.4 s
Wall time: 23.4 s


In [87]:
%%time
prior_block_res+i

CPU times: user 10.8 ms, sys: 202 ms, total: 213 ms
Wall time: 214 ms


array([[79.93955803, 79.93955803, 79.93955803, ..., 67.00354906,
        67.00354906, 67.00354906],
       [78.12277429, 78.12277429, 78.12277429, ..., 65.71694239,
        65.71694239, 65.71694239],
       [78.08883397, 78.08883397, 78.08883397, ..., 66.16226338,
        66.16226338, 66.16226338],
       ...,
       [78.27077827, 78.27077827, 78.27077827, ..., 66.21571485,
        66.21571485, 66.21571485],
       [79.77113025, 79.77113025, 79.77113025, ..., 65.44080601,
        65.44080601, 65.44080601],
       [78.76716189, 78.76716189, 78.76716189, ..., 64.84271099,
        64.84271099, 64.84271099]])

In [ ]:
### Playing around with functions for mean calculation ...
@njit
def mean_calc(a):
    """
    Input numpy array
    """
    bs=a.shape[0]
    l=a.shape[1]
    #
    m=np.empty(l)
    for i in range(l):
        m[i]=a[:,i].mean() 
    return m

#explicitely parallelized
@njit(parallel=True)
def mean_calc_p(a):
    """
    Input numpy array
    """
    bs=a.shape[0]
    l=a.shape[1]
    #
    m=np.empty(l)
    for i in prange(l):
        m[i]=a[:,i].mean() 
        #m[i]=a[:,i].sum()/bs
    return m

#explicitely fastmath
@njit(fastmath=True)
def mean_calc_fm(a):
    """
    Input numpy array
    """
    bs=a.shape[0]
    l=a.shape[1]
    #
    m=np.empty(l)
    for i in range(l):
        m[i]=a[:,i].mean() 
    return m

#explicitely fastmath
@vectorize(target="parallel")
def mean_calc_vec(a):
    """
    Input numpy array
    """
    bs=a.shape[0]
    l=a.shape[1]
    #
    m=np.empty(l)
    for i in prange(l):
        m[i]=a[:,i].mean() 
    return m